# library

In [1]:
!pip install sastrawi -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.7/209.7 kB 3.1 MB/s eta 0:00:00


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import requests

# Load Data

In [3]:
data = pd.read_csv('/content/review_exhuma.csv', delimiter=';')
data.head()

,created_at,id_str,full_text,quote_count,reply_count,retweet_count,favorite_count,lang,user_id_str,conversation_id_str,username,tweet_url
0,Mon Mar 18 05:47:33 +0000 2024,1769601514685989175,hari jumat kemaren akhirnya nonton exhuma dan ...,1,3,0,0,in,1347584151256408067,1769601514685989175,kaizkdlinnie,https://twitter.com/kaizkdlinnie/status/176960...
1,Mon Mar 18 03:11:02 +0000 2024,1769562127554220230,Macam tempted pula mau tgk exhuma di wayang sb...,0,0,0,0,in,3916294700,1769562127554220230,jxlvinjeneral,https://twitter.com/jxlvinjeneral/status/17695...
2,Mon Mar 18 02:39:41 +0000 2024,1769554236453335354,📝NOL DERAJAT FILM REVIEW📝 Judul Film : Exhuma...,0,1,0,5,in,804453042,1769554236453335354,nol_derajat,https://twitter.com/nol_derajat/status/1769554...
3,Sun Mar 17 21:55:01 +0000 2024,1769482600421474553,Semalem akhirnya bisa nonton EXHUMA. Yang mau ...,0,0,0,0,in,130407343,1769482600421474553,viracitra,https://twitter.com/viracitra/status/176948260...
4,Sun Mar 17 17:38:24 +0000 2024,1769418019988803672,"Review singkat Exhuma 2024. 7,5/10. Rekomen bu...",0,3,0,2,in,1167307244196618241,1769418019988803672,stakeaxetion,https://twitter.com/stakeaxetion/status/176941...


In [4]:
# Delete the specified columns
columns_to_delete = ['created_at', 'id_str', 'reply_count', 'quote_count',
                     'retweet_count', 'favorite_count', 'lang', 'user_id_str',
                     'conversation_id_str', 'username', 'tweet_url']

data.drop(columns=columns_to_delete, inplace=True)

# Display the updated DataFrame
data.head()

,full_text
0,hari jumat kemaren akhirnya nonton exhuma dan ...
1,Macam tempted pula mau tgk exhuma di wayang sb...
2,📝NOL DERAJAT FILM REVIEW📝 Judul Film : Exhuma...
3,Semalem akhirnya bisa nonton EXHUMA. Yang mau ...
4,"Review singkat Exhuma 2024. 7,5/10. Rekomen bu..."


# Text Processing

In [5]:
def cleaning_text(text):
    # remove url
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    text = url_pattern.sub(r'', text)

    # remove hashtags
    # only removing the hash # sign from the word
    text = re.sub(r'#', '', text)

    # remove mention handle user (@)
    text = re.sub(r'@[\w]*', ' ', text)

    # remove emojis
    emoji_pattern = re.compile(
        '['
        '\U0001F600-\U0001F64F'  # emoticons
        '\U0001F300-\U0001F5FF'  # symbols & pictographs
        '\U0001F680-\U0001F6FF'  # transport & map symbols
        '\U0001F700-\U0001F77F'  # alchemical symbols
        '\U0001F780-\U0001F7FF'  # Geometric Shapes Extended
        '\U0001F800-\U0001F8FF'  # Supplemental Arrows-C
        '\U0001F900-\U0001F9FF'  # Supplemental Symbols and Pictographs
        '\U0001FA00-\U0001FA6F'  # Chess Symbols
        '\U0001FA70-\U0001FAFF'  # Symbols and Pictographs Extended-A
        '\U00002702-\U000027B0'  # Dingbats
        '\U000024C2-\U0001F251'
        ']+',
        flags=re.UNICODE
    )
    text = emoji_pattern.sub(r'', text)

    # remove punctuation
    punctuations = '''!()-[]{};:'"\,<>./?@#$%^&*_~'''
    for x in text.lower():
        if x in punctuations:
            text = text.replace(x, " ")

    # remove extra whitespace
    text = ' '.join(text.split())

    # lowercase
    text = text.lower()
    return text

import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')

# CONSTRUCT STOPWORDS
rama_stopword = "https://raw.githubusercontent.com/ramaprakoso/analisis-sentimen/master/kamus/stopword.txt"
yutomo_stopword = "https://raw.githubusercontent.com/yasirutomo/python-sentianalysis-id/master/data/feature_list/stopwordsID.txt"
fpmipa_stopword = "https://raw.githubusercontent.com/onlyphantom/elangdev/master/elang/word2vec/utils/stopwords-list/fpmipa-stopwords.txt"
sastrawi_stopword = "https://raw.githubusercontent.com/onlyphantom/elangdev/master/elang/word2vec/utils/stopwords-list/sastrawi-stopwords.txt"
aliakbar_stopword = "https://raw.githubusercontent.com/onlyphantom/elangdev/master/elang/word2vec/utils/stopwords-list/aliakbars-bilp.txt"
pebahasa_stopword = "https://raw.githubusercontent.com/onlyphantom/elangdev/master/elang/word2vec/utils/stopwords-list/pebbie-pebahasa.txt"
elang_stopword = "https://raw.githubusercontent.com/onlyphantom/elangdev/master/elang/word2vec/utils/stopwords-id.txt"
nltk_stopword = stopwords.words('indonesian')

# create path url for each stopword
path_stopwords = [rama_stopword, yutomo_stopword, fpmipa_stopword, sastrawi_stopword,
                  aliakbar_stopword, pebahasa_stopword, elang_stopword]

# combine stopwords
stopwords_l = nltk_stopword
for path in path_stopwords:
    response = requests.get(path)
    stopwords_l += response.text.split('\n')

custom_st = '''
yg yang dgn ane smpai bgt gua gwa si tu ama utk udh btw
ntar lol ttg emg aj aja tll sy sih kalo nya trsa mnrt nih
ma dr ajaa tp akan bs bikin kta pas pdahl bnyak guys abis tnx
bang banget nang mas amat bangettt tjoy hemm haha sllu hrs lanjut
bgtu sbnrnya trjadi bgtu pdhl sm plg skrg
'''

# create dictionary with unique stopword
st_words = set(stopwords_l)
custom_stopword = set(custom_st.split())

# result stopwords
stop_words = st_words | custom_stopword
print(f'Stopwords: {list(stop_words)[:5]}')
# remove stopwords
from nltk import word_tokenize, sent_tokenize

def remove_stopword(text, stop_words=stop_words):
    word_tokens = word_tokenize(text)
    filtered_sentence = [w for w in word_tokens if not w in stop_words]
    return ' '.join(filtered_sentence)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Stopwords: ['keadaan', 'berapapun', 'bisa', 'kiranya', 'dah']


In [6]:
# pipeline preprocess
def preprocess(text):
    # cleaning text and lowercase
    output = cleaning_text(text)

    # remove stopwords
    output = remove_stopword(output)

    return output

In [7]:
# implement preprocessing
preprocessed_data = data.copy()
preprocessed_data['full_text'] = data['full_text'].map(preprocess)

In [8]:
df = pd.DataFrame(preprocessed_data)
csv_file_path = 'exhuma_preprocessed_data.csv'
df.to_csv(csv_file_path, sep=';', index=False, header=True)

print(f'Data has been saved to {csv_file_path}')

Data has been saved to exhuma_preprocessed_data.csv


In [9]:
# load dataset into pandas
import pandas as pd
data = pd.read_csv('/content/exhuma_preprocessed_data.csv', sep=';')
data

,full_text
0,kemaren nonton exhuma setuju review moots2ku k...
1,tempted tgk exhuma wayang sbb review bnyk ckp ...
2,nol derajat film review judul film exhuma rili...
3,semalem nonton exhuma nonton baiknya baca2 rev...
4,review singkat exhuma 2024 7 5 10 rekomen teme...
...,...
636,exhuma raih rating sempurna cgv golden egg ind...
637,•kdm• review singkat film exhuma roh jeongeui ...
638,tiktoker suka review drakor film invite wawanc...
639,exhuma bioskop review letterboxd
